# Lambda Calculus: PyCon 2019


link: https://www.youtube.com/watch?v=5C6sv7-eTKg

Dude, don't ask me why pls

In [1]:
from functools import partial

In [2]:
printl = partial(print, end="\n"*2)

In [3]:
# switch bev.
def LEFT(a):
    def f(b):
        return a
    return f

def RIGTH(a):
    def f(b):
        return b
    return f

In [4]:
print(LEFT("left")("right"))

print(RIGTH("left")("right"))

left
right


In [7]:
# boolean
def TRUE(x):
    return lambda y: x

def FALSE(x):
    return lambda y: y

# reverse values
def NOT(x):
    return x(FALSE)(TRUE)

# AND & OR
def AND(x):
    """
    2 and 3 -> 3
    0 and 3 -> 0
    """
    return lambda y: x(y)(x)

def OR(x):
    """
    2 or 3 -> 2
    0 or 3 -> 3
    """
    return lambda y: x(x)(y)

In [10]:
# logical true and false
print("True return left: ", TRUE("true")("false"))
printl("False return right: ", FALSE("true")("false"))

# logical not
print("assert not True == False: ", NOT(TRUE))
printl("assert not False == True: ", NOT(FALSE))

print("NOT(TRUE): ", NOT(TRUE)("true")("false"))
printl("NOT(FALSE): ", NOT(FALSE)("true")("false"))

# logical and & or
print(AND(TRUE)(TRUE), AND(TRUE)(FALSE), AND(FALSE)(TRUE), AND(FALSE)(FALSE))
printl(OR(TRUE)(TRUE), OR(TRUE)(FALSE), OR(FALSE)(TRUE), OR(FALSE)(FALSE))

True return left:  true
False return right:  false

assert not True == False:  <function FALSE at 0x10243a268>
assert not False == True:  <function TRUE at 0x10437ae18>

NOT(TRUE):  false
NOT(FALSE):  true

<function TRUE at 0x10437ae18> <function FALSE at 0x10243a268> <function FALSE at 0x10243a268> <function FALSE at 0x10243a268>
<function TRUE at 0x10437ae18> <function TRUE at 0x10437ae18> <function TRUE at 0x10437ae18> <function FALSE at 0x10243a268>



In [211]:
# Church numerals 
ZERO = lambda f: lambda x: x
ONE = lambda f: lambda x: f(x)
TWO = lambda f: lambda x: f(f(x))
THREE = lambda f: lambda x: f(f(f(x)))
FOUR = lambda f: lambda x: f(f(f(f(x))))

INC = lambda x: x + 1

# SUCC(TWO) -> THREE
SUCC = lambda n: (lambda f: lambda x: f(n(f)(x)))

# ADD & MULT

# ADD(TWO)(THREE) -> FIVE | TWO(SUCC)(TWO)(INC)(0)
ADD = lambda x: lambda y: x(SUCC)(y)

# MULT(TWO)(THREE) -> SIX | TWO(TWO(INC))(0)
MULT = lambda x: lambda y: (lambda f: y(x(f)))
POW = lambda x: lambda y: (lambda f: y(x)(f))

# paired data sctruct
CONS = lambda x: lambda y: lambda f: f(x)(y)
CAR = lambda p: p(TRUE)
CDR = lambda p: p(FALSE)

# PREV(TWO) -> ONE
T = lambda p: CONS(SUCC(CAR(p)))(CAR(p)) 
PRED = lambda n: CDR(n(T)(CONS(ZERO)(ZERO)))

# minus
SUB = lambda x: lambda y: y(PRED)(x)

# test number for zero
ISZERO = lambda n: n(lambda f: FALSE)(TRUE)

# factorial function
FACT = lambda n: (ISZERO(n)
                      (ONE)
                      (MULT(n)(FACT(PRED(n)))))


# fix Python args evaluation and inf rec
LAZY_TRUE = lambda x: lambda y: x()
LAZY_FALSE = lambda x: lambda y: y()
LAZY_ISZERO = lambda n: n(lambda f: LAZY_FALSE)(LAZY_TRUE)

LAZY_FACT = lambda n: (LAZY_ISZERO(n)
                          (lambda: ONE)
                          (lambda: MULT(n)(LAZY_FACT(PRED(n)))))


In [126]:
p = CONS(2)(3)
print(CAR(p), CDR(p))

2 3


In [152]:
a = THREE(T)(CONS(ZERO)(ZERO))

In [154]:
CAR(a)(INC)(0)

3

In [195]:
a = POW(THREE)(FOUR)
printl("3^4: ", a(INC)(0))

b = PRED(a)
print("3^4 - 1: ", b(INC)(0))

3^4:  81

3^4 - 1:  80


In [192]:
print("3 - 2 = ", SUB(THREE)(TWO)(INC)(0))

3 - 2 =  1


In [198]:
ISZERO(ZERO)

<function __main__.TRUE(x)>

In [203]:
FACT(ONE)

RecursionError: maximum recursion depth exceeded

In [214]:
LAZY_FACT(FOUR)(INC)(0)

24

# Y-Combinator (wow magick)

Problem: we cant refer function itself inside function by name! they are all anonymous!

In [225]:
LAZY_FACT = lambda f: lambda n: (LAZY_ISZERO(n)
                          (lambda: ONE)
                          (lambda: MULT(n)(f(PRED(n)))))

In [233]:
R = (lambda f: lambda n: 1 if n==0 else n*f(n-1))
Y = lambda f: (lambda x: f(x(x)))(lambda x: f(x(x)))
# Python fix
Y_wow = lambda f: (lambda x: f(lambda z: x(x)(z)))(lambda x: f(lambda z: x(x)(z)))

In [234]:
fact_Y = Y_wow(R)

In [242]:
fact_Y(4)

24

In [236]:
fact_lambda = Y_wow(LAZY_FACT)

In [241]:
fact_lambda(FOUR)(INC)(0)

24

## holy crap!!!!